<a href="https://colab.research.google.com/github/gurovic/MLCourse/blob/main/325_creating_features.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Создание признаков (Feature Engineering)

**Цель:**  
Научиться преобразовывать исходные данные в информативные признаки, улучшающие качество ML-моделей.

## 🟢 Базовые методы генерации признаков

### Полиномиальные признаки (для линейных моделей)  
**Зачем:** Линейные модели (линейная регрессия, логистическая регрессия) плохо улавливают нелинейные зависимости. Полиномиальные признаки помогают решить эту проблему.  

**Пример:**  
Дан признак \( x \). Добавляем \( x^2, x^3 \) и т. д.  

**Код:**

In [5]:
from sklearn.preprocessing import PolynomialFeatures

X = [[2], [3], [4]]  # Исходный признак
poly = PolynomialFeatures(degree=2, include_bias=False)
X_poly = poly.fit_transform(X)  # → [[2, 4], [3, 9], [4, 16]]

**Когда использовать:**  
- Линейные модели  
- Данные с неочевидными нелинейными зависимостями  

**⚠️ Осторожно:**  
- Может привести к переобучению при высокой степени (`degree`)  
- Увеличивает размерность данных

### Взаимодействие признаков (Feature Interaction)  
**Зачем:** Учесть комбинированное влияние нескольких признаков.  

**Пример:**  
Даны признаки \( x_1 \) (возраст) и \( x_2 \) (доход). Добавляем \( x_1 \times x_2 \).  

**Код:**

In [6]:
import pandas as pd

df = pd.DataFrame({"age": [25, 30, 35], "income": [50000, 60000, 70000]})
df["age_income_interaction"] = df["age"] * df["income"]  # Новый признак

**Когда использовать:**  
- Есть предположение, что признаки влияют на целевую переменную совместно  
- В линейных моделях и деревьях

## 🟡 Агрегаты для реляционных данных  
**Зачем:** Если данные связаны (например, транзакции пользователей), можно создать признаки на основе статистик.  

**Пример:**  
Дан датасет транзакций. Для каждого пользователя считаем:  
- Средний чек  
- Количество транзакций  
- Максимальный/минимальный платеж  

**Код:**

In [7]:
transactions = pd.DataFrame({
    "user_id": [1, 1, 2, 2, 2],
    "amount": [100, 150, 200, 50, 300]
})

# Агрегация по пользователю
user_stats = transactions.groupby("user_id")["amount"].agg(["mean", "count", "max"])

**Когда использовать:**  
- Данные с иерархией (пользователи → транзакции)  
- Для задач классификации/регрессии, где важны групповые статистики

## 🔴 Признаки из дат и времени  
**Зачем:** Даты содержат скрытые паттерны (сезонность, тренды).  

### Извлечение признаков из даты:  
| Признак          | Пример значения |  
|------------------|----------------|  
| Год (`year`)     | 2023           |  
| Месяц (`month`)  | 12 (декабрь)   |  
| День недели (`dayofweek`) | 2 (вторник) |  
| Часы (`hour`)    | 15 (3 PM)      |  

**Код:**

In [8]:
# Создаем тестовый DataFrame с датами
data = {
    "transaction_id": [1, 2, 3, 4, 5],
    "date_column": [
        "2023-01-15 08:30:00",
        "2023-02-20 14:45:00",
        "2023-03-10 09:15:00",
        "2023-04-05 18:20:00",
        "2023-05-12 11:10:00"
    ],
    "value": [100, 200, 150, 300, 250]
}

df = pd.DataFrame(data)

# Преобразование в datetime с явным указанием формата
df["datetime"] = pd.to_datetime(
    df["date_column"],
    format="%Y-%m-%d %H:%M:%S"  # Явный формат для надежности
)

# Извлечение компонентов даты
df["year"] = df["datetime"].dt.year
df["month"] = df["datetime"].dt.month
df["day"] = df["datetime"].dt.day
df["hour"] = df["datetime"].dt.hour
df["minute"] = df["datetime"].dt.minute
df["day_of_week"] = df["datetime"].dt.dayofweek  # 0-6 (пн-вс)
df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)

# Дополнительные признаки
df["time_of_day"] = pd.cut(
    df["hour"],
    bins=[0, 6, 12, 18, 24],
    labels=["Ночь", "Утро", "День", "Вечер"],
    right=False
)

# Проверка результата
print(df[["datetime", "year", "month", "day_of_week", "is_weekend", "time_of_day"]].head())

             datetime  year  month  day_of_week  is_weekend time_of_day
0 2023-01-15 08:30:00  2023      1            6           1        Утро
1 2023-02-20 14:45:00  2023      2            0           0        День
2 2023-03-10 09:15:00  2023      3            4           0        Утро
3 2023-04-05 18:20:00  2023      4            2           0       Вечер
4 2023-05-12 11:10:00  2023      5            4           0        Утро


### Дополнительные идеи:  
- **Флаг выходного дня:**  
  ```python
  df["is_weekend"] = df["day_of_week"].isin([5, 6]).astype(int)
  ```
- **Время суток (утро/день/вечер):**  
  ```python
  df["time_of_day"] = pd.cut(df["hour"], bins=[0, 6, 12, 18, 24], labels=["ночь", "утро", "день", "вечер"])
  ```

**Когда использовать:**  
- Прогнозирование временных рядов  
- Анализ сезонности (например, продажи, спрос)

## 📊 Чеклист по уровням  
| Уровень | Навыки | Инструменты |  
|---------|--------|-------------|  
| **🟢** | Полиномы, взаимодействие признаков | `PolynomialFeatures`, умножение признаков |  
| **🟡** | Агрегация по группам | `groupby()`, `agg()` |  
| **🔴** | Извлечение признаков из дат | `pandas.to_datetime()`, `.dt` |

## 💡 Полезные практики  
1. **Автоматизация:** Используйте `FeatureUnion` и `Pipeline` в `sklearn` для удобного добавления признаков.  
2. **Визуализация:** Стройте графики (`pairplot`, `heatmap`), чтобы проверять новые признаки.  
3. **Отбор признаков:** Удаляйте бесполезные признаки с помощью `SelectKBest` или `VIF`.

## 📌 Итог  
1. **Полиномы** улучшают линейные модели.  
2. **Агрегаты** помогают в задачах с реляционными данными.  
3. **Даты** можно разложить на информативные компоненты.  

> **Философия главы:**  
> *«Хорошие признаки важнее сложных моделей. Генерация признаков — это творческий процесс, требующий понимания данных.»*  

**🔜 Следующая глава:** Отбор признаков и уменьшение размерности.